In [1]:
# https://pystan.readthedocs.io/en/latest/getting_started.html

import nest_asyncio
nest_asyncio.apply()

import stan

schools_code = """
data {
  int<lower=0> J;         // number of schools
  real y[J];              // estimated treatment effects
  real<lower=0> sigma[J]; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

In [2]:
schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

In [3]:
posterior = stan.build(schools_code, data=schools_data, random_seed=1)

Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_3k73g5yz/model_deucahs6.stan', line 4, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_3k73g5yz/model_deucahs6.stan', line 5, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.


In [4]:
fit = posterior.sample(num_chains=4, num_samples=1000)

Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 3.6e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.36 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 9.3e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.93 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3.7e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.37 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 4.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.41 seconds.
  Adjust your expectations accordingly!


In [5]:
eta = fit["eta"]  # array with shape (8, 4000)
eta

array([[-0.84251155,  1.39450154,  1.72554834, ..., -0.91534058,
         0.46738558,  0.11077997],
       [ 1.67972792,  0.60052647,  0.27564521, ...,  0.21171247,
         0.71718797, -1.89614611],
       [ 1.93201433,  1.42027688,  0.00272407, ...,  0.7802034 ,
        -0.32021488, -0.78735738],
       ...,
       [ 0.8277197 ,  0.11208344,  0.26635756, ..., -1.41730966,
         0.39467637, -1.56029185],
       [ 0.0609284 ,  1.47795746, -0.80011264, ..., -1.58942072,
         0.44392529,  1.38158421],
       [ 0.19894895, -0.45791237,  0.63196456, ..., -1.23898409,
         0.5354359 ,  0.12082592]])

In [6]:
df = fit.to_frame()
df

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,mu,tau,eta.1,...,eta.7,eta.8,theta.1,theta.2,theta.3,theta.4,theta.5,theta.6,theta.7,theta.8
draws,,,,,,,,,,,,,,,,,,,,,
0,-41.434797,0.995804,0.329328,4.0,15.0,0.0,49.613906,1.411805,3.712575,-0.842512,...,0.060928,0.198949,-1.716083,7.647921,8.584553,0.118586,0.255418,4.484776,1.638006,2.150418
1,-40.351307,0.960118,0.362729,3.0,7.0,0.0,43.888786,5.565448,5.241893,1.394502,...,1.477957,-0.457912,12.875275,8.713343,13.010387,12.390397,11.624891,6.152977,13.312742,3.165120
2,-38.787739,0.707885,0.339015,4.0,15.0,0.0,42.791248,9.089039,4.088504,1.725548,...,-0.800113,0.631965,16.143950,10.216015,9.100176,9.355503,13.088015,10.178043,5.817775,11.672828
3,-38.070688,0.993421,0.276670,4.0,15.0,0.0,41.100882,-3.252739,15.142525,1.790945,...,1.302802,0.271314,23.866685,-4.558579,5.296715,15.118712,-13.721916,1.606133,16.474977,0.855648
4,-39.943404,0.976922,0.329328,3.0,15.0,0.0,49.458689,0.988765,8.718704,2.116322,...,0.570918,0.800307,19.440349,9.779775,-17.787455,6.800586,3.661638,-2.470068,5.966430,7.966407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-38.938803,0.804256,0.276670,4.0,15.0,0.0,42.685992,3.600862,7.030774,0.308411,...,-0.994636,0.220432,5.769229,12.568509,3.097115,-1.263883,6.676914,6.571913,-3.392196,5.150672
3996,-40.248194,0.993653,0.329328,4.0,15.0,0.0,46.562869,-1.210811,2.353864,-0.001527,...,0.111574,0.763645,-1.214406,-2.317131,-0.754548,-4.186838,-0.233024,-2.351499,-0.948181,0.586705
3997,-41.561686,0.965625,0.362729,3.0,7.0,0.0,46.382168,4.097240,2.958960,-0.915341,...,-1.589421,-1.238984,1.388784,4.723689,6.405831,3.501403,1.671126,-0.096523,-0.605793,0.431135
